In [22]:
import numpy as np
from matplotlib import pyplot as plt
from matplotlib.ticker import AutoMinorLocator
from mayavi import mlab
from numba import jit
from tvtk.tools import visual
from sympy import *
from sympy import sqrt
from sympy import cos, sin
x2, y2, z2 = symbols("x2 y2 z2")
x1, y1, z1 = symbols("x1 y1 z1")
import mpmath

In [23]:
#the angle between the rotation axis and the observer 
delta = 79*np.pi/180.0

T = 5
omega = 2*np.pi/T

R = 1e6
r_em = 4*R

N1 = 100
N2 = 100
N3 = 100

w=5e6
xa = np.linspace(-w,w,N1)
ya = np.linspace(-w,w,N2)
za = np.linspace(-w,w,N3)
center = int(N1/2.0)

px = 0.0
py = 0.985
pz = 0.174

In [24]:
#the angle between the magnetic moment axis and the observer
alpha = 80*np.pi/180.0

#the angle between the rotation axis and the observer 
delta = 79*np.pi/180.0

def cal_m(phi):
    phi = phi+np.pi/2.0
    m_x = np.sin(alpha)*np.cos(delta)*np.cos(phi)-np.sin(delta)*np.cos(alpha)
    m_y = np.sin(alpha)*np.sin(phi)
    m_z = np.cos(alpha)*np.cos(delta) + np.sin(alpha)*np.sin(delta)*np.cos(phi)
    return m_x,m_y,m_z
         
def cal_bfield(phi_new, th_em, phi_em, r_em, h):
    x_ray = r_em*np.sin(th_em)*np.cos(phi_em)
    y_ray = r_em*np.sin(th_em)*np.sin(phi_em)
    z_ray = r_em*np.cos(th_em) + h
    m_x, m_y, m_z = cal_m(phi_new)
    r = np.sqrt(x_ray**2 + y_ray**2 + z_ray**2)
    B_y = (-m_y/r**3 + 3*y_ray*(m_x*x_ray + m_y*y_ray + m_z*z_ray)/r**5)
    B_x = (-m_x/r**3 + 3*x_ray*(m_x*x_ray + m_y*y_ray + m_z*z_ray)/r**5)
    B_z = (-m_z/r**3 + 3*z_ray*(m_x*x_ray + m_y*y_ray + m_z*z_ray)/r**5)
    Bmag = np.sqrt(B_x**2+B_y**2+B_z**2)
    return B_x/Bmag, B_y/Bmag

def get_emission_point_dip(m_x,m_y,m_z):
    th_em = np.arccos(m_z)/2.0 - 0.5*np.arcsin(0.33*np.sin(np.arccos(m_z)))
    phi_em = np.arctan(m_y/m_x)
    #print (th_em*180/np.pi, phi_em*180/np.pi)
    return th_em, phi_em


def get_bfield(x,y,z,phi):
    m_x, m_y, m_z = cal_m(phi)
    r = np.sqrt(x**2 + y**2 + z**2)
    B_y = (-m_y/r**3 + 3*y*(m_x*x + m_y*y + m_z*z)/r**5)
    B_x = (-m_x/r**3 + 3*x*(m_x*x + m_y*y + m_z*z)/r**5)
    B_z = (-m_z/r**3 + 3*z*(m_x*x + m_y*y + m_z*z)/r**5)
    Bmag = np.sqrt(B_x**2+B_y**2+B_z**2)
    return B_x/Bmag, B_y/Bmag, B_z/Bmag

def get_field(x0,y0,z0,t):
    x = np.cos(omega*t)*(x0*np.cos(delta) + z0*np.sin(delta))+np.sin(omega*t)*y0
    y = -np.sin(omega*t)*(np.cos(delta)*x0 + np.sin(delta)*z0)+ np.cos(omega*t)*y0
    z = -x0*np.sin(delta) + z0*np.cos(delta)
    r1 = np.sqrt(x**2+y**2+z**2)
    r2 = np.sqrt(x**2+y**2+(z-0.4*R)**2)
    rqrt = 0.6*x**2 + y*(-0.8*y-2*(z-0.4*R)) + (z-0.4*R)*(-2*y+0.2*(z-0.4*R))
    #rqrt = 0.6*x**2 + y*(-0.8*y-2*(z+0.4)) + (z+0.4)*(-2*y+0.2*(z+0.4))
    bxd = -px/r1**3 + 3*(px*x+py*y+pz*z)*x/r1**5
    byd = -py/r1**3 + 3*(px*x+py*y+pz*z)*y/r1**5
    bzd = -pz/r1**3 + 3*(px*x+py*y+pz*z)*z/r1**5
    bxq = -2*0.6*x/r2**5 + 5.0*(rqrt)*x/r2**7
    byq = -2*(-0.8*y-2*(z-0.4*R))/r2**5 + 5.0*(rqrt)*y/r2**7
    bzq = -2*(-2*y+0.2*(z-0.4*R))/r2**5 + 5.0*(rqrt)*(z-0.4*R)/r2**7
    bx = bxd+bxq
    by = byd+byq
    bz = bzd+bzq
    Bx = np.cos(delta)*np.cos(omega*t)*bx - np.cos(delta)*np.sin(omega*t)*by-np.sin(delta)*bz
    By = np.sin(omega*t)*bx + np.cos(omega*t)*by
    Bz = np.sin(delta)*np.cos(omega*t)*bx - np.sin(delta)*np.sin(omega*t)*by + np.cos(delta)*bz
    Bmag = np.sqrt(Bx**2+By**2+Bz**2)
    return Bx/Bmag, By/Bmag, Bz/Bmag

def Analytical_field(t):
    x2 = np.cos(omega*t)*(x1*np.cos(delta) + z1*np.sin(delta))+np.sin(omega*t)*y1
    y2 = -np.sin(omega*t)*(np.cos(delta)*x1+np.sin(delta)*z1)+ np.cos(omega*t)*y1
    z2 = -x1*np.sin(delta) + z1*np.cos(delta)
    r1 = sqrt(x2**2+y2**2+z2**2)
    r2 = sqrt(x2**2+y2**2+(z2-0.4*R)**2)
    rqrt = 0.6*x2**2 + y2*(-0.8*y2-2*(z2-0.4*R)) + (z2-0.4*R)*(-2*y2+0.2*(z2-0.4*R))
    #rqrt = 0.6*x**2 + y*(-0.8*y-2*(z+0.4)) + (z+0.4)*(-2*y+0.2*(z+0.4))
    bxd = -px/r1**3 + 3*(px*x2+py*y2+pz*z2)*x2/r1**5
    byd = -py/r1**3 + 3*(px*x2+py*y2+pz*z2)*y2/r1**5
    bzd = -pz/r1**3 + 3*(px*x2+py*y2+pz*z2)*z2/r1**5
    bxq = -2*0.6*x2/r2**5 + 5.0*(rqrt)*x2/r2**7
    byq = -2*(-0.8*y2-2*(z2-0.4*R))/r2**5 + 5.0*(rqrt)*y2/r2**7
    bzq = -2*(-2*y2+0.2*(z2-0.4*R))/r2**5 + 5.0*(rqrt)*(z2-0.4*R)/r2**7
    bx = bxd+bxq
    by = byd+byq
    bz = bzd+bzq
    Bx = np.cos(delta)*np.cos(omega*t)*bx - np.cos(delta)*np.sin(omega*t)*by-np.sin(delta)*bz
    By = np.sin(omega*t)*bx + np.cos(omega*t)*by
    Bz = np.sin(delta)*np.cos(omega*t)*bx - np.sin(delta)*np.sin(omega*t)*by + np.cos(delta)*bz
    Bmag = sqrt(Bx**2+By**2+Bz**2)
    return Bx/Bmag, By/Bmag, Bz/Bmag


def cal_PA_dip(h,r_em,T,ax):
    c = 3e10
    t_ray = h*1e3/c
    t = 0.0
    dt = 0.01
    Rl = c*T*1e-3/2/np.pi
    PA, phase = [],[]
    phi = 0
    while phi<360*np.pi/180:
        phi = 2*np.pi*t/T
        m_x, m_y, m_z = cal_m(phi)
        th_em, phi_em = get_emission_point_dip(m_x,m_y,m_z)
        B_x, B_y = cal_bfield(phi + 2*np.pi*t_ray/T, th_em, phi_em, r_em, h)
        Psi = np.arctan(B_y/B_x)
        PA.append(Psi)
        phase.append(phi)
        t += dt
    PA = np.array(PA)*180/np.pi
    phase = np.array(phase)*180/np.pi
    ax.plot(phase,PA,label=r"h="+str(round(h*1e-5))+" km")

In [25]:
def rgrid():
    r = np.zeros((N1,N2,N3))
    for i in range(N1):
        for j in range(N2):
            for k in range(N3):
                r[i][j][k] = np.sqrt(xa[i]**2+ya[j]**2+za[k]**2)
    return r
r = rgrid()

i_id,j_id,k_id = [],[],[]
for i in range(N1):
    for j in range(N2):
        for k in range(N3):
            if r[i][j][k]>1*r_em and r[i][j][k]<1.05*r_em:
                #print (r[i][j][k])
                i_id.append(i)
                j_id.append(j)
                k_id.append(k)               

i_id = np.array(i_id)
j_id = np.array(j_id)
k_id = np.array(k_id)


eps = 0.01

def get_temp_emission_point(t):
    x_t_em,y_t_em,z_t_em = [],[],[]
    for p in range(len(i_id)):
        b_x, b_y, b_z = get_field(xa[i_id[p]],ya[j_id[p]],za[k_id[p]],t)
        if np.abs(b_x)<0.1 and np.abs(b_y)<0.1 and za[k_id[p]]>0:
            x_t_em.append(xa[i_id[p]])
            y_t_em.append(ya[j_id[p]])
            z_t_em.append(za[k_id[p]])
    return x_t_em,y_t_em,z_t_em

In [26]:
def get_emission_point(t,gx,gy,gz):
    a1,a2,a3 = Analytical_field(t)
    eq3 = sqrt(x1**2+y1**2+z1**2-r_em**2)
    roots = nsolve((a1, a2, eq3), (x1, y1, z1), (gx,gy,gz), verify=False)
    xem = float(roots[0])
    yem = float(roots[1])
    zem = float(roots[2])   
    print ("field =",get_field(xem,yem,zem,t))
    return xem,yem,zem

In [27]:
def compute_total_field(t):
    b1 = np.zeros((N1,N2,N3))
    b2 = np.zeros((N1,N2,N3))
    b3 = np.zeros((N1,N2,N3))
    for i in range(N1):
        for j in range(N2):
            for k in range(N3):
                bv1,bv2,bv3 = get_field(xa[i],ya[j],za[k],t)
                if (xa[i]**2+ya[j]**2+za[k]**2>1):
                    b1[i][j][k] = bv1
                    b2[i][j][k] = bv2
                    b3[i][j][k] = bv3
    bmag = np.sqrt(b1**2+b2**2+b3**2)
    return b1,b2,b3

In [28]:
thetas = np.linspace(0,np.pi,100)
phis = np.linspace(0,2*np.pi,100)
xsurf = np.zeros((len(thetas),len(phis)))
ysurf = np.zeros((len(thetas),len(phis)))
zsurf = np.zeros((len(thetas),len(phis)))
Bsurf = np.zeros((len(thetas),len(phis)))
for i in range(len(thetas)):
    for j in range(len(phis)):
        xsurf[i][j] = R*np.sin(thetas[i])*np.cos(phis[j])
        ysurf[i][j] = R*np.sin(thetas[i])*np.sin(phis[j])
        zsurf[i][j] = R*np.cos(thetas[i])      
        bx_s,by_s,bz_s = get_field(xsurf[i][j],ysurf[i][j],zsurf[i][j],0)
        if bz_s>0 and zsurf[i][j]>0:
            Bsurf[i][j] = bz_s  

In [29]:
def drawSphereMlab(center, radius, **kwargs):
    xc, yc, zc = center
    phi, theta = np.mgrid[0:np.pi:101j, 0:2*np.pi:101j]
    x = xc + radius * np.sin(phi) * np.cos(theta)
    y = yc + radius * np.sin(phi) * np.sin(theta)
    z = zc + radius * np.cos(phi)
    sphere = mlab.mesh(x, y, z, **kwargs)
    return sphere

In [30]:
s = Bsurf
v1,v2,v3 = compute_total_field(0)

In [31]:
fig = mlab.figure(1, size=(800, 800), bgcolor=(1, 1, 1), fgcolor=(0, 0, 0))
field = mlab.pipeline.vector_field(v1, v2, v3)  
magnitude = mlab.pipeline.extract_vector_norm(field)
field_lines = mlab.pipeline.streamline(magnitude, seedtype='sphere',
                                       integration_direction='both',colormap='bone')
drawSphereMlab((center,center,center), 2, color=(0,0,1))
#a=visual.Arrow(x=5,y=5,z=5)
#mlab.clf()
#mlab.mesh(xsurf, ysurf, zsurf, scalars=s, colormap='jet')
#mlab.view(azimuth=1, elevation=7, distance=10.0)
#field_lines.seed.widget.point1 = [1e6, 0, 0]
field_lines.stream_tracer.maximum_propagation = 1000.
#field_lines.seed.widget.point1 = [69, 75.5, 75.5]
#field_lines.seed.widget.point2 = [82, 75.5, 75.5]
#field_lines.seed.widget.resolution = 50
field_lines.seed.widget.enabled = False
mlab.savefig(filename="%s.png"%str(0))
mlab.show()

qt.qpa.window: <QNSWindow: 0x7fc49af716e0; contentView=<QNSView: 0x7fc49af70fb0; QCocoaWindow(0x7fc49af70ea0, window=QWidgetWindow(0x7fc49af709f0, name="QMainWindowClassWindow"))>> has active key-value observers (KVO)! These will stop working now that the window is recreated, and will result in exceptions when the observers are removed. Break in QCocoaWindow::recreateWindowIfNeeded to debug.


In [ ]:
#@jit(parallel=True)
def cal_newbfield(xem,yem,zem,h,time):
    x_ray = xem
    y_ray = yem
    z_ray = zem + h
    B_x,B_y,B_z = get_field(x_ray,y_ray,z_ray,time)
    return B_x, B_y

In [ ]:
#@jit(parallel=True)
def cal_PA(h,T,ax,color,lb):
    c = 3e10
    t_ray = h*1e3/c
    t = 0
    dt = 0.3
    Rl = c*T*1e-3/2/np.pi
    PA, phase = [],[]
    phi = 0
    while phi<360/180*np.pi:
        phi = omega*(t)
        print (t, phi*180/np.pi)
        if t<=3.0 or t>4:
            xt, yt, zt = get_temp_emission_point(t)
            try:
                xem,yem,zem = get_emission_point(t,xt[0],yt[0],zt[0])
                if zem>0.0:
                    B_x, B_y = cal_newbfield(xem,yem,zem,h,t+t_ray)
                    Psi = np.arctan(B_y/B_x)
                    PA.append(Psi)
                    phase.append(phi)
            except:
                pass
            t = t+dt
        if t>3 or t<=4:
            try:
                xem,yem,zem = get_emission_point(t,xt[1],yt[1],zt[1])
                B_x, B_y = cal_newbfield(xem,yem,zem,h,t+t_ray)
                Psi = np.arctan(B_y/B_x)
                PA.append(Psi)
                phase.append(phi)
            except:
                pass
            t = t+0.01
        
        '''
        for i in xr:
            for j in xr:
                try:
                    xem,yem,zem = get_emission_point(t,i,np.abs(j),i)
                    B_x, B_y = cal_newbfield(xem,yem,zem,h,t+t_ray)
                    Psi = np.arctan(B_y/B_x)
                    PA.append(Psi)
                    phase.append(phi)                        
                except:
                    pass
        
            
            if (np.abs(bxt)>0.01 or np.abs(byt)>0.01):
                t+=dt
                continue
            else:
                B_x, B_y = cal_newbfield(xem,yem,zem,h,t+t_ray)
                Psi = np.arctan(B_y/B_x)
                PA.append(Psi)
                phase.append(phi)
            '''
    PA = np.array(PA)*180/np.pi
    phase = np.array(phase)*180/np.pi
    ax.scatter(phase,PA,s=20,c=color,alpha=0.8,label="r_em = "+str(lb/R)+"R")
    #ax.plot(phase,PA)

In [ ]:
f = plt.figure(figsize=(8,7))
ax = f.add_subplot(111)
cal_PA_dip(1e5,r_em, T, ax)
cal_PA(1e5, T, ax, 'gray',r_em)
ax.set_xlabel(r'$\phi$[deg]',size=18)
ax.set_ylabel(r'P.A. ($\Psi$ [deg])',size=18)
ax.legend(loc="best",fontsize=14,ncol=1)
ax.xaxis.set_minor_locator(AutoMinorLocator())
ax.yaxis.set_minor_locator(AutoMinorLocator())
ax.tick_params(axis="x", labelsize=18) 
ax.tick_params(axis="y", labelsize=18)
ax.tick_params(axis='both', which='major', direction='in',labelsize=10, length=4, width=1., colors='k', left=True, right=True, top=True, pad=8)
ax.tick_params(axis='both', which='minor', direction='in',labelsize=10, length=2, width=1, colors='k', left=True, right=True, top=True, pad=8)
plt.savefig("PA_quad_dip.png",bbox_inches='tight')
plt.show()

In [ ]:
array1 = np.linspace(0,2*np.pi,100)
array2 = array1*T/2/np.pi
np.savetxt('t_phi.txt', np.array([array1*180/np.pi, array2]).T, delimiter='\t', fmt="%s")

In [ ]:
x, y, z = np.ogrid[- .5:.5:200j, - .5:.5:200j, - .5:.5:200j]
r = np.sqrt(x ** 2 + y ** 2 + z ** 2)


# Generalized Laguerre polynomial (3, 2)
L = - r ** 3 / 6 + 5. / 2 * r ** 2 - 10 * r + 6

# Spherical harmonic (3, 2)
Y = (x + y * 1j) ** 2 * z / r ** 3

Phi = L * Y * np.exp(- r) * r ** 2

# Plot it ####################################################################
from mayavi import mlab
mlab.figure(1, fgcolor=(1, 1, 1), bgcolor=(0, 0, 0))
# We create a scalar field with the module of Phi as the scalar
src = mlab.pipeline.scalar_field(np.abs(Phi))

# And we add the phase of Phi as an additional array
# This is a tricky part: the layout of the new array needs to be the same
# as the existing dataset, and no checks are performed. The shape needs
# to be the same, and so should the data. Failure to do so can result in
# segfaults.
src.image_data.point_data.add_array(np.angle(Phi).T.ravel())
# We need to give a name to our new dataset.
src.image_data.point_data.get_array(1).name = 'angle'
# Make sure that the dataset is up to date with the different arrays:
src.update()

# We select the 'scalar' attribute, ie the norm of Phi
src2 = mlab.pipeline.set_active_attribute(src,
                                    point_scalars='scalar')

# Cut isosurfaces of the norm
contour = mlab.pipeline.contour(src2)

# Now we select the 'angle' attribute, ie the phase of Phi
contour2 = mlab.pipeline.set_active_attribute(contour,
                                    point_scalars='angle')

# And we display the surface. The colormap is the current attribute: the phase.
mlab.pipeline.surface(contour2, colormap='hsv')

mlab.colorbar(title='Phase', orientation='vertical', nb_labels=3)

mlab.show()